# Rețele neuronale recurente

În modulul anterior, am utilizat reprezentări semantice bogate ale textului și un clasificator liniar simplu deasupra acestor încorporări. Ceea ce face această arhitectură este să capteze semnificația agregată a cuvintelor dintr-o propoziție, dar nu ia în considerare **ordinea** cuvintelor, deoarece operația de agregare aplicată încorporărilor elimină această informație din textul original. Deoarece aceste modele nu pot modela ordonarea cuvintelor, ele nu pot rezolva sarcini mai complexe sau ambigue, cum ar fi generarea de text sau răspunsul la întrebări.

Pentru a capta semnificația unei secvențe de text, trebuie să utilizăm o altă arhitectură de rețea neuronală, numită **rețea neuronală recurentă**, sau RNN. În RNN, trecem propoziția prin rețea, simbol cu simbol, iar rețeaua produce un anumit **stat**, pe care îl trecem din nou prin rețea împreună cu următorul simbol.

Dată fiind secvența de tokeni $X_0,\dots,X_n$, RNN creează o secvență de blocuri de rețea neuronală și antrenează această secvență cap-coadă folosind propagarea înapoi. Fiecare bloc de rețea primește o pereche $(X_i,S_i)$ ca intrare și produce $S_{i+1}$ ca rezultat. Starea finală $S_n$ sau ieșirea $X_n$ este transmisă unui clasificator liniar pentru a produce rezultatul. Toate blocurile de rețea împărtășesc aceleași greutăți și sunt antrenate cap-coadă folosind o singură trecere de propagare înapoi.

Deoarece vectorii de stare $S_0,\dots,S_n$ sunt trecuți prin rețea, aceasta este capabilă să învețe dependențele secvențiale dintre cuvinte. De exemplu, atunci când cuvântul *not* apare undeva în secvență, rețeaua poate învăța să nege anumite elemente din vectorul de stare, rezultând o negare.

> Deoarece greutățile tuturor blocurilor RNN din imagine sunt împărtășite, aceeași imagine poate fi reprezentată ca un singur bloc (în dreapta) cu o buclă de feedback recurent, care transmite starea de ieșire a rețelei înapoi la intrare.

Să vedem cum rețelele neuronale recurente ne pot ajuta să clasificăm setul nostru de date de știri.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Clasificator RNN simplu

În cazul unui RNN simplu, fiecare unitate recurentă este o rețea liniară simplă, care primește un vector de intrare concatenat și un vector de stare, și produce un nou vector de stare. PyTorch reprezintă această unitate prin clasa `RNNCell`, iar o rețea formată din astfel de celule - prin stratul `RNN`.

Pentru a defini un clasificator RNN, vom aplica mai întâi un strat de încorporare pentru a reduce dimensiunea vocabularului de intrare, iar apoi vom adăuga un strat RNN deasupra acestuia:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** Folosim un strat de încorporare neantrenat aici pentru simplitate, dar pentru rezultate și mai bune putem folosi un strat de încorporare pre-antrenat cu încorporări Word2Vec sau GloVe, așa cum este descris în unitatea anterioară. Pentru o înțelegere mai bună, s-ar putea să doriți să adaptați acest cod pentru a funcționa cu încorporări pre-antrenate.

În cazul nostru, vom folosi un loader de date cu secvențe completate, astfel încât fiecare lot va avea un număr de secvențe completate de aceeași lungime. Stratul RNN va lua secvența de tensori de încorporare și va produce două ieșiri:
* $x$ este o secvență de ieșiri ale celulelor RNN la fiecare pas
* $h$ este starea ascunsă finală pentru ultimul element al secvenței

Aplicăm apoi un clasificator liniar complet conectat pentru a obține numărul de clase.

> **Note:** RNN-urile sunt destul de dificile de antrenat, deoarece odată ce celulele RNN sunt desfășurate de-a lungul lungimii secvenței, numărul rezultat de straturi implicate în propagarea înapoi este destul de mare. Astfel, trebuie să selectăm o rată de învățare mică și să antrenăm rețeaua pe un set de date mai mare pentru a obține rezultate bune. Poate dura destul de mult timp, așa că utilizarea GPU-ului este preferată.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Memorie pe Termen Lung și Scurt (LSTM)

Una dintre principalele probleme ale RNN-urilor clasice este așa-numita problemă a **gradientului care dispare**. Deoarece RNN-urile sunt antrenate cap-coadă într-o singură trecere de back-propagation, acestea întâmpină dificultăți în propagarea erorii către primele straturi ale rețelei, iar astfel rețeaua nu poate învăța relațiile dintre tokenii distanți. Una dintre modalitățile de a evita această problemă este introducerea **gestionării explicite a stării** prin utilizarea așa-numitelor **porți**. Există două arhitecturi bine cunoscute de acest tip: **Memorie pe Termen Lung și Scurt** (LSTM) și **Unitatea de Releu cu Porți** (GRU).

![Imagine care arată un exemplu de celulă LSTM](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Rețeaua LSTM este organizată într-un mod similar cu RNN, dar există două stări care sunt transmise de la un strat la altul: starea actuală $c$ și vectorul ascuns $h$. La fiecare unitate, vectorul ascuns $h_i$ este concatenat cu intrarea $x_i$, iar acestea controlează ce se întâmplă cu starea $c$ prin intermediul **porților**. Fiecare poartă este o rețea neuronală cu activare sigmoidă (ieșire în intervalul $[0,1]$), care poate fi considerată ca o mască bit cu bit atunci când este înmulțită cu vectorul de stare. Există următoarele porți (de la stânga la dreapta în imaginea de mai sus):
* **Poarta de uitare** preia vectorul ascuns și determină ce componente ale vectorului $c$ trebuie să uităm și care să fie transmise mai departe.
* **Poarta de intrare** preia informații din intrare și din vectorul ascuns și le inserează în stare.
* **Poarta de ieșire** transformă starea printr-un strat liniar cu activare $\tanh$, apoi selectează unele dintre componentele sale folosind vectorul ascuns $h_i$ pentru a produce noua stare $c_{i+1}$.

Componentele stării $c$ pot fi considerate ca niște semnale care pot fi activate sau dezactivate. De exemplu, când întâlnim un nume precum *Alice* într-o secvență, putem presupune că se referă la un personaj feminin și putem activa semnalul în stare care indică faptul că avem un substantiv feminin în propoziție. Când întâlnim ulterior expresia *și Tom*, vom activa semnalul care indică faptul că avem un substantiv la plural. Astfel, prin manipularea stării, putem, teoretic, să urmărim proprietățile gramaticale ale părților propoziției.

> **Note**: O resursă excelentă pentru a înțelege detaliile interne ale LSTM este acest articol minunat [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) de Christopher Olah.

Deși structura internă a celulei LSTM poate părea complexă, PyTorch ascunde această implementare în clasa `LSTMCell` și oferă obiectul `LSTM` pentru a reprezenta întregul strat LSTM. Astfel, implementarea unui clasificator LSTM va fi destul de similară cu cea a unui RNN simplu pe care l-am văzut mai sus:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Secvențe împachetate

În exemplul nostru, a trebuit să completăm toate secvențele din minibatch cu vectori de zero. Deși acest lucru duce la o risipă de memorie, în cazul RNN-urilor este mai critic faptul că se creează celule RNN suplimentare pentru elementele de intrare completate, care participă la antrenament, dar nu conțin informații importante. Ar fi mult mai bine să antrenăm RNN doar pentru dimensiunea reală a secvenței.

Pentru a face acest lucru, PyTorch introduce un format special de stocare a secvențelor completate. Să presupunem că avem un minibatch de intrare completat care arată astfel:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Aici, 0 reprezintă valorile completate, iar vectorul de lungime reală al secvențelor de intrare este `[5,3,1]`.

Pentru a antrena eficient RNN cu secvențe completate, dorim să începem antrenamentul primului grup de celule RNN cu un minibatch mare (`[1,6,9]`), dar apoi să terminăm procesarea celei de-a treia secvențe și să continuăm antrenamentul cu minibatch-uri mai mici (`[2,7]`, `[3,8]`) și așa mai departe. Astfel, secvența împachetată este reprezentată ca un singur vector - în cazul nostru `[1,6,9,2,7,3,8,4,5]`, și un vector de lungime (`[5,3,1]`), din care putem reconstrui cu ușurință minibatch-ul completat original.

Pentru a produce o secvență împachetată, putem folosi funcția `torch.nn.utils.rnn.pack_padded_sequence`. Toate straturile recurente, inclusiv RNN, LSTM și GRU, acceptă secvențe împachetate ca intrare și produc o ieșire împachetată, care poate fi decodificată folosind `torch.nn.utils.rnn.pad_packed_sequence`.

Pentru a putea produce o secvență împachetată, trebuie să transmitem vectorul de lungime rețelei, și astfel avem nevoie de o funcție diferită pentru a pregăti minibatch-urile:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Rețeaua efectivă ar fi foarte similară cu `LSTMClassifier` de mai sus, dar trecerea `forward` va primi atât minibatch-ul umplut, cât și vectorul lungimilor secvențelor. După ce calculăm embedding-ul, calculăm secvența împachetată, o trecem prin stratul LSTM și apoi despachetăm rezultatul înapoi.

> **Notă**: De fapt, nu folosim rezultatul despachetat `x`, deoarece utilizăm ieșirea din straturile ascunse în calculele următoare. Astfel, putem elimina complet despachetarea din acest cod. Motivul pentru care o plasăm aici este pentru a vă permite să modificați acest cod cu ușurință, în cazul în care aveți nevoie să utilizați ieșirea rețelei în calcule ulterioare.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Notă:** Este posibil să fi observat parametrul `use_pack_sequence` pe care îl transmitem funcției de antrenare. În prezent, funcția `pack_padded_sequence` necesită ca tensorul de lungime a secvenței să fie pe dispozitivul CPU, și astfel funcția de antrenare trebuie să evite mutarea datelor de lungime a secvenței pe GPU în timpul antrenării. Puteți analiza implementarea funcției `train_emb` în fișierul [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## RNN-uri bidirecționale și multilayer

În exemplele noastre, toate rețelele recurente au funcționat într-o singură direcție, de la începutul unei secvențe până la sfârșit. Pare natural, deoarece seamănă cu modul în care citim și ascultăm vorbirea. Totuși, în multe cazuri practice, avem acces aleatoriu la secvența de intrare, așa că ar putea avea sens să efectuăm calculul recurent în ambele direcții. Astfel de rețele se numesc **RNN-uri bidirecționale**, și pot fi create prin transmiterea parametrului `bidirectional=True` către constructorul RNN/LSTM/GRU.

Când lucrăm cu o rețea bidirecțională, vom avea nevoie de două vectori de stare ascunsă, câte unul pentru fiecare direcție. PyTorch codifică acești vectori ca un singur vector de dimensiune dublă, ceea ce este destul de convenabil, deoarece, de obicei, vectorul de stare rezultat este transmis unui strat liniar complet conectat, și trebuie doar să ținem cont de această creștere în dimensiune atunci când creăm stratul.

O rețea recurentă, fie unidirecțională, fie bidirecțională, captează anumite modele dintr-o secvență și le poate stoca în vectorul de stare sau le poate transmite în ieșire. La fel ca în cazul rețelelor convoluționale, putem construi un alt strat recurent deasupra primului pentru a capta modele de nivel superior, construite din modelele de nivel inferior extrase de primul strat. Acest lucru ne conduce la conceptul de **RNN multilayer**, care constă din două sau mai multe rețele recurente, unde ieșirea stratului anterior este transmisă stratului următor ca intrare.

![Imagine care arată un RNN multilayer cu memorie pe termen lung și scurt](../../../../../translated_images/ro/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Imagine din [această postare minunată](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) de Fernando López*

PyTorch face construirea unor astfel de rețele o sarcină ușoară, deoarece trebuie doar să transmiteți parametrul `num_layers` către constructorul RNN/LSTM/GRU pentru a construi automat mai multe straturi de recurență. Acest lucru înseamnă, de asemenea, că dimensiunea vectorului de stare ascunsă va crește proporțional, și va trebui să țineți cont de acest aspect atunci când gestionați ieșirea straturilor recurente.


## RNN-uri pentru alte sarcini

În această unitate, am văzut că RNN-urile pot fi utilizate pentru clasificarea secvențelor, dar, de fapt, ele pot gestiona multe alte sarcini, cum ar fi generarea de text, traducerea automată și altele. Vom analiza aceste sarcini în unitatea următoare.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa maternă ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
